In [95]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Data Preprocessing
image_dir = r"C:/Users/HOME/dacpvsrn"
image_size = (100, 100)  # Resize all images to 100x100

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized)
    return np.array(images)

# Load images
images = load_images_from_folder(image_dir)

# Normalize images
images = images / 255.0

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create fake labels for now (you can modify this part with actual labels)
labels = np.random.randint(0, 10, len(images))  # Assume 10 classes

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Flatten images for PCA
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# 2. PCA-based Face Recognition
pca = PCA(n_components=0.95, svd_solver='full')  # Capture 95% variance
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

# Simple Neural Network for classification using PCA components
model_pca = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_pca.shape[1],)),
    Dropout(0.3),  # Regularization with dropout
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Adjust based on the number of classes
])

# Compile with a lower learning rate for better convergence
model_pca.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation during training
history_pca = model_pca.fit(datagen.flow(X_train_pca, y_train, batch_size=32), 
                            validation_data=(X_test_pca, y_test), epochs=30)

# Evaluation
y_pred_pca = np.argmax(model_pca.predict(X_test_pca), axis=-1)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
precision_pca = precision_score(y_test, y_pred_pca, average='weighted')
recall_pca = recall_score(y_test, y_pred_pca, average='weighted')
f1_pca = f1_score(y_test, y_pred_pca, average='weighted')

# 3. Neural Network-based Face Recognition (No PCA)
model_nn = Sequential([
    Flatten(input_shape=image_size),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Adjust based on the number of classes
])

# Compile model with similar optimizer adjustments
model_nn.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the neural network model
history_nn = model_nn.fit(datagen.flow(X_train, y_train, batch_size=32), 
                          validation_data=(X_test, y_test), epochs=30)

# Evaluation
y_pred_nn = np.argmax(model_nn.predict(X_test), axis=-1)
accuracy_nn = accuracy_score(y_test, y_pred_nn)
precision_nn = precision_score(y_test, y_pred_nn, average='weighted')
recall_nn = recall_score(y_test, y_pred_nn, average='weighted')
f1_nn = f1_score(y_test, y_pred_nn, average='weighted')

# 4. Comparison of results
print("PCA-based Model Metrics:")
print(f"Accuracy: {accuracy_pca:.2f}, Precision: {precision_pca:.2f}, Recall: {recall_pca:.2f}, F1 Score: {f1_pca:.2f}")

print("\nNeural Network-based Model Metrics:")
print(f"Accuracy: {accuracy_nn:.2f}, Precision: {precision_nn:.2f}, Recall: {recall_nn:.2f}, F1 Score: {f1_nn:.2f}")

# Plot loss and accuracy over epochs for both models
plt.figure(figsize=(12, 5))

# PCA Model
plt.subplot(1, 2, 1)
plt.plot(history_pca.history['accuracy'], label='PCA Model Training Accuracy')
plt.plot(history_pca.history['val_accuracy'], label='PCA Model Validation Accuracy')
plt.title('PCA Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# NN Model
plt.subplot(1, 2, 2)
plt.plot(history_nn.history['accuracy'], label='NN Model Training Accuracy')
plt.plot(history_nn.history['val_accuracy'], label='NN Model Validation Accuracy')
plt.title('NN Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


C:\Users\HOME\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape (112, 67)